In [1]:
import json
import requests
from tqdm import tqdm
from xml.etree import ElementTree

url = "http://localhost:9003/?output_format=jsonl"

text = "A mogę, bo moim zdaniem jest do niczego. I to delikatnie mówiąc... Rzecz gustu :) Ja sobie z przyjemnością obejrzałam wczoraj \"Live and Let Die\". Nareszcie bez Connery'ego!!! Pasuje. Najbardziej ze wszystkich. Może trochę za ładny, ale grą pasuje. IMHO. A Connery to nie święta krowa ;)"

In [2]:
def parse_line(line):
    r = requests.post(url, data = line.encode('utf-8'))
    out = json.loads(r.text)
    return out

flatten = lambda t: [item for sublist in t for item in sublist]

In [3]:
with open('test-raw.txt') as f:
    lines = list(filter(lambda x: x != "", [line.rstrip() for line in f]))
tree = ElementTree.parse('test-gold.xml')
root = tree.getroot()

In [4]:
chunks = tree.find("chunkList").findall("chunk")

matched = 0
miss_matched = 0
CTAG_POSSITION = 2
BASE_POSSITION = 1
TEXT_POSSITION = 0

chunk_id = 0
for line in tqdm(chunks):
#     print(len(curr_line), curr_line)
    
    for sentence in line.findall("chunk"):
        sentence_text =  [word.find("orth").text for word in sentence.findall("tok")]
        if len(sentence_text) == 0:
            continue
        curr_line = flatten(parse_line(' '.join(sentence_text)))
        word_id = 0
        
        for word in sentence.findall("tok"):
            if word_id >= len(curr_line):
                miss_matched += 1
                break
            
            curr_word = curr_line[word_id]
            orth = word.find("orth").text
            base = word.find("lex").find("base").text
            ctag = word.find("lex").find("ctag").text

#             print(f"{orth} {base} {ctag}")
#             print(curr_word)
            if ctag == curr_word[CTAG_POSSITION]:
                matched += 1
            else:
#                 print(orth, ctag)
#                 print(curr_word[TEXT_POSSITION], curr_word[CTAG_POSSITION])
                miss_matched += 1
            word_id += 1
            
    chunk_id += 1

100%|██████████| 209/209 [00:40<00:00,  5.20it/s]


In [5]:
print(matched, miss_matched)

15290 12273


In [6]:
matched/(matched+miss_matched)

0.5547291659108224